In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn import metrics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.metrics import pairwise_distances
from sklearn.metrics import silhouette_score
from sklearn.metrics import davies_bouldin_score


In [19]:
df = pd.read_csv('data/DRUG1n.csv') 
df.head()

,Age,Sex,BP,Cholesterol,Na,K,Drug
0,23,F,HIGH,HIGH,0.792535,0.031258,drugY
1,47,M,LOW,HIGH,0.739309,0.056468,drugC
2,47,M,LOW,HIGH,0.697269,0.068944,drugC
3,28,F,NORMAL,HIGH,0.563682,0.072289,drugX
4,61,F,LOW,HIGH,0.559294,0.030998,drugY


In [20]:
df.shape

(200, 7)

In [21]:
#查找缺失值
df.isnull().sum()

Age            0
Sex            0
BP             0
Cholesterol    0
Na             0
K              0
Drug           0
dtype: int64

分类特征数值化以及one-hot编码

In [22]:
# F : 0, M: 1
df['Sex'] = df['Sex'].map({'F':0,'M':1})
df['Sex'].unique()

array([0, 1], dtype=int64)

In [23]:
# HIGH : 0 , NORMAL: 1
df['Cholesterol'] = df['Cholesterol'].map({'HIGH':0,'NORMAL':1})
df['Cholesterol'].unique()

array([0, 1], dtype=int64)

In [24]:
for i in ['BP','Drug']:
    dummies = pd.get_dummies(df[i])
    df = pd.concat([df,dummies],axis =1)
    df.drop(i,axis = 1,inplace = True)
df = df.rename(columns={'HIGH': 'BP_HIGH'})
df = df.rename(columns={'LOW': 'BP_LOW'})
df = df.rename(columns={'NORMAL': 'BP_NORMAL'})
df.head()

,Age,Sex,Cholesterol,Na,K,BP_HIGH,BP_LOW,BP_NORMAL,drugA,drugB,drugC,drugX,drugY
0,23,0,0,0.792535,0.031258,1,0,0,0,0,0,0,1
1,47,1,0,0.739309,0.056468,0,1,0,0,0,1,0,0
2,47,1,0,0.697269,0.068944,0,1,0,0,0,1,0,0
3,28,0,0,0.563682,0.072289,0,0,1,0,0,0,1,0
4,61,0,0,0.559294,0.030998,0,1,0,0,0,0,0,1


In [25]:
df.shape

(200, 13)

In [26]:
# # 设置标签 用来标记聚类结果
# df['class'] = np.zeros([df.shape[0]])
# # 设置列表 用来确定 哪些坐标是聚类中心
# k_center = {}
# #准备可视化的列 三个数值列
# # temp = df[['Age','K','class']]
df.head()

,Age,Sex,Cholesterol,Na,K,BP_HIGH,BP_LOW,BP_NORMAL,drugA,drugB,drugC,drugX,drugY
0,23,0,0,0.792535,0.031258,1,0,0,0,0,0,0,1
1,47,1,0,0.739309,0.056468,0,1,0,0,0,1,0,0
2,47,1,0,0.697269,0.068944,0,1,0,0,0,1,0,0
3,28,0,0,0.563682,0.072289,0,0,1,0,0,0,1,0
4,61,0,0,0.559294,0.030998,0,1,0,0,0,0,0,1


In [27]:
std = StandardScaler()
data = std.fit_transform(df)
data.shape

(200, 13)

In [36]:
# DBSCAN模型
dbscan = DBSCAN(eps=1.2, min_samples=2)
labels = dbscan.fit_predict(data)

# 计算每个簇的簇内平方和（WCSS）
unique_labels = np.unique(labels)
wcss_values = []

for label in unique_labels:
    if label == -1:
        continue  # 跳过噪声点
    cluster_points = data[labels == label]
    wcss = np.sum(pairwise_distances(cluster_points, metric='euclidean')**2) / (2 * len(cluster_points))
    wcss_values.append(wcss)

# 输出簇标签和簇内平方和
print("Cluster Labels:", labels)
print("WCSS Values for Each Cluster:", wcss_values[-1])

   Age  Sex  Cholesterol        Na         K  BP_HIGH  BP_LOW  BP_NORMAL  \
0   23    0            0  0.792535  0.031258        1       0          0   
1   47    1            0  0.739309  0.056468        0       1          0   
2   47    1            0  0.697269  0.068944        0       1          0   
3   28    0            0  0.563682  0.072289        0       0          1   
4   61    0            0  0.559294  0.030998        0       1          0   

   drugA  drugB  drugC  drugX  drugY  Cluster  AvgDistance  
0      0      0      0      0      1       -1     3.600746  
1      0      0      1      0      0       -1     3.600746  
2      0      0      1      0      0       -1     3.600746  
3      0      0      0      1      0       -1     3.600746  
4      0      0      0      0      1       -1     3.600746  
Cluster                    -1          0          1
Age         count  183.000000  12.000000   5.000000
            mean    45.573770  26.000000  42.200000
            std     1